In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                type_ = html.xpath('//span[@id="lblPartTypeName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                                
                application_summary = ''.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip().replace(', ', '\n')

                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//a[@id="ProductPic"]/@href')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
                if len(vehicle_url) == 0:
                    vehicle_url = ''
                else:
                    vehicle_url = 'https://www.dormanproducts.com/' + vehicle_url[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_oe = [oe.strip() for oe in html.xpath('//section[@id="productOE"]/descendant::tbody/tr/th/text()')]
                list_oe = list(set(list_oe))
                oe = ';'.join(list_oe)
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//section[@id="productSpec"]/descendant::tr')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Part_Number': part_number,
                                         'Type': type_,
                                         'Description': description,
                                         'Application_Summary': application_summary,
                                         'Vehicle_1': '',
                                         'Vehicle_2': '',
                                         'OE': oe,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Vehicle_Url': vehicle_url,
                                         'Src': src}])

                for tr in list_tr:
                    name = tr.xpath('./th/text()')[0].lower().strip()[:-1]
                    df_temp.loc[0, name[:-1].strip() if name.endswith(':') else name] = tr.xpath('./td/text()')[0].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：393

[ok] - https://www.dormanproducts.com/p-134355-574-120.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：373] - [当前时间：13:59:30]

[ok] - https://www.dormanproducts.com/p-132253-574-014.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：372] - [当前时间：13:59:30]

[ok] - https://www.dormanproducts.com/p-134353-574-109.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：371] - [当前时间：13:59:30]

[ok] - https://www.dormanproducts.com/p-134350-574-091.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：370] - [当前时间：13:59:30]

[ok] - https://www.dormanproducts.com/p-140889-574-079.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：369] - [当前时间：13:59:30]

[ok] - https://www.dormanproducts.com/p-133436-574-050.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：368] - [当前时间：13:59:30]

[ok] - https://www.dormanproducts.com/p-140157-574-085.aspx?parttype=Fuel%2520Filler%2520Ne

[ok] - https://www.dormanproducts.com/p-84603-577-339.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：318] - [当前时间：13:59:37]

[ok] - https://www.dormanproducts.com/p-52616-577-097.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：317] - [当前时间：13:59:38]

[ok] - https://www.dormanproducts.com/p-84445-577-356.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：316] - [当前时间：13:59:38]

[ok] - https://www.dormanproducts.com/p-84292-577-363.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：315] - [当前时间：13:59:39]

[ok] - https://www.dormanproducts.com/p-74902-577-297.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：2] - [剩余数量：314] - [当前时间：13:59:39]

[ok] - https://www.dormanproducts.com/p-73537-577-233.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：313] - [当前时间：13:59:39]

[ok] - https://www.dormanproducts.com/p-133947-577-156.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keywo

[ok] - https://www.dormanproducts.com/p-49003-577-940.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：6] - [剩余数量：261] - [当前时间：13:59:51]

[ok] - https://www.dormanproducts.com/p-132252-574-011.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：4] - [剩余数量：260] - [当前时间：13:59:51]

[ok] - https://www.dormanproducts.com/p-139630-574-221.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：259] - [当前时间：13:59:51]

[ok] - https://www.dormanproducts.com/p-132258-574-026.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：13] - [剩余数量：258] - [当前时间：13:59:52]

[ok] - https://www.dormanproducts.com/p-134345-574-074.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：12] - [剩余数量：257] - [当前时间：13:59:52]

[ok] - https://www.dormanproducts.com/p-134364-574-237.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：3] - [剩余数量：256] - [当前时间：13:59:52]

[ok] - https://www.dormanproducts.com/p-44275-577-926.aspx?parttype=Fuel%2520Filler%2520Neck&origin

[ok] - https://www.dormanproducts.com/p-134352-574-101.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：205] - [当前时间：14:00:04]

[ok] - https://www.dormanproducts.com/p-134344-574-070.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：204] - [当前时间：14:00:05]

[ok] - https://www.dormanproducts.com/p-85544-577-379.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：6] - [剩余数量：203] - [当前时间：14:00:05]

[ok] - https://www.dormanproducts.com/p-73948-577-221.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：5] - [剩余数量：202] - [当前时间：14:00:05]

[ok] - https://www.dormanproducts.com/p-83953-577-345.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：6] - [剩余数量：201] - [当前时间：14:00:05]

[ok] - https://www.dormanproducts.com/p-50477-577-084.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：18] - [剩余数量：200] - [当前时间：14:00:06]

[ok] - https://www.dormanproducts.com/p-135291-574-046.aspx?parttype=Fuel%2520Filler%2520Neck&origin=ke

[ok] - https://www.dormanproducts.com/p-134383-577-154.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：3] - [剩余数量：149] - [当前时间：14:00:20]

[ok] - https://www.dormanproducts.com/p-5212-577-958.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：17] - [剩余数量：148] - [当前时间：14:00:20]

[ok] - https://www.dormanproducts.com/p-134361-574-170.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：25] - [剩余数量：147] - [当前时间：14:00:21]

[ok] - https://www.dormanproducts.com/p-75281-577-255.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：146] - [当前时间：14:00:21]

[ok] - https://www.dormanproducts.com/p-133946-577-142.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：3] - [剩余数量：145] - [当前时间：14:00:21]

[ok] - https://www.dormanproducts.com/p-75327-577-288.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：3] - [剩余数量：144] - [当前时间：14:00:21]

[ok] - https://www.dormanproducts.com/p-48782-577-946.aspx?parttype=Fuel%2520Filler%2520Neck&origin=ke

[ok] - https://www.dormanproducts.com/p-134347-574-077.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：6] - [剩余数量：93] - [当前时间：14:00:34]

[ok] - https://www.dormanproducts.com/p-132358-574-013.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：92] - [当前时间：14:00:35]

[ok] - https://www.dormanproducts.com/p-132366-574-030.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：8] - [剩余数量：91] - [当前时间：14:00:35]

[ok] - https://www.dormanproducts.com/p-141536-574-189.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：7] - [剩余数量：90] - [当前时间：14:00:35]

[ok] - https://www.dormanproducts.com/p-73920-577-225.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：11] - [剩余数量：89] - [当前时间：14:00:35]

[ok] - https://www.dormanproducts.com/p-134346-574-076.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：88] - [当前时间：14:00:35]

[ok] - https://www.dormanproducts.com/p-133440-574-055.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keywo

[ok] - https://www.dormanproducts.com/p-73523-577-252.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：5] - [剩余数量：36] - [当前时间：14:00:48]

[ok] - https://www.dormanproducts.com/p-75239-577-286.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：35] - [当前时间：14:00:48]

[ok] - https://www.dormanproducts.com/p-84452-577-949.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：4] - [剩余数量：34] - [当前时间：14:00:48]

[ok] - https://www.dormanproducts.com/p-76154-577-293.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：2] - [剩余数量：33] - [当前时间：14:00:48]

[ok] - https://www.dormanproducts.com/p-75100-577-229.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：1] - [剩余数量：32] - [当前时间：14:00:49]

[ok] - https://www.dormanproducts.com/p-84542-577-359.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次数：5] - [剩余数量：31] - [当前时间：14:00:49]

[ok] - https://www.dormanproducts.com/p-84965-577-316.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword
[尝试次

KeyError: 'No.'